In [ ]:
# Install Mambaforge using curl and bash
system('curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh', intern = TRUE)
system('bash Mambaforge-Linux-x86_64.sh -b -u -p $HOME/mambaforge', intern = TRUE)

# Add mambaforge bin to the system path
Sys.setenv(PATH = paste(Sys.getenv("HOME"), "/mambaforge/bin:", Sys.getenv("PATH"), sep = ""))

In [ ]:
# Install gsutil and dependencies using mamba
system('mamba install -y -c conda-forge -c bioconda gsutil', intern = TRUE)

In [ ]:
# Install BiocManager if not installed
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

# Set repositories
options(repos = BiocManager::repositories())

# Install Bioconductor packages
BiocManager::install(c("ComplexHeatmap", "DESeq2", "edgeR"), force = TRUE)

# Install CRAN packages
install.packages(c("dplyr", "pheatmap", "ggrepel", "ggfortify", "devtools"), dependencies = TRUE)

In [ ]:
# Load libraries
library(DESeq2)
library(dplyr)
library(ComplexHeatmap)
library(edgeR)
library(ggplot2)
library(ggrepel)
library(ggfortify)
library(devtools)

In [ ]:
# Install NetAct
install_github("lusystemsbio/NetAct", dependencies=T, build_vignettes = T)

In [ ]:
library(NetAct)

#url <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63310&format=file"
#utils::download.file(url, destfile="GSE63310_RAW.tar", mode="wb") 
#utils::untar("GSE63310_RAW.tar", exdir = ".")
#files <- c("GSM1545535_10_6_5_11.txt", "GSM1545536_9_6_5_11.txt", "GSM1545538_purep53.txt",
#           "GSM1545539_JMS8-2.txt", "GSM1545540_JMS8-3.txt", "GSM1545541_JMS8-4.txt",
#           "GSM1545542_JMS8-5.txt", "GSM1545544_JMS9-P7c.txt", "GSM1545545_JMS9-P8c.txt")
#for(i in paste(files, ".gz", sep="")){
#  R.utils::gunzip(i, overwrite=TRUE)
#}

#files <- c("IAV06_IAV12_read_counts.txt")

files <- c("GSM1545535_10_6_5_11.txt", "GSM1545536_9_6_5_11.txt", 
           "GSM1545538_purep53.txt", "GSM1545539_JMS8-2.txt", 
           "GSM1545540_JMS8-3.txt", "GSM1545541_JMS8-4.txt", 
           "GSM1545542_JMS8-5.txt", "GSM1545544_JMS9-P7c.txt", 
           "GSM1545545_JMS9-P8c.txt")

x <- edgeR::readDGE(files, columns=c(1,3))
group <- as.factor(c("LP", "ML", "Basal", "Basal", "ML", "LP", "Basal", "ML", "LP"))
x$samples$group <- group
samplenames <- c("LP1", "ML1", "Basal1", "Basal2", "ML2", "LP2", "Basal3", "ML3", "LP3")
colnames(x) <- samplenames

In [ ]:
compList <- c("Basal-LP", "Basal-ML", "LP-ML")
phenoData = new("AnnotatedDataFrame", data = data.frame(celltype = group))
rownames(phenoData) = colnames(x$counts)

In [ ]:
counts <- Preprocess_counts(counts = x$counts, groups = group, mouse = TRUE)
DErslt = RNAseqDegs_limma(counts = counts, phenodata = phenoData, 
                          complist = compList, qval = 0.05)

In [ ]:
DErslt$Overall

In [ ]:
neweset = Biobase::ExpressionSet(assayData = as.matrix(DErslt$Overall$e), phenoData = phenoData)

In [ ]:
DErslt$Overall$e

In [ ]:
DErslt2 = RNAseqDegs_DESeq(counts = counts, phenodata = phenoData, 
                          complist = compList, qval = 0.05)
neweset2 = Biobase::ExpressionSet(assayData = as.matrix(DErslt2$Overall$e), phenoData = phenoData)


In [ ]:
data("mDB")
calc <- TRUE

if (calc) {
  gsearslts <- TF_Selection(GSDB = mDB, DErslt = DErslt, minSize = 5, nperm = 10000,
                            qval = 0.05, compList = compList,
                            nameFile = "gsearslts_tutorial")
} else {
  gsearslts <- readRDS(file = "gsearslts_tutorial.RDS")
}

tfs <- gsearslts$tfs
tfs

In [ ]:
Reselect_TFs(GSEArslt = gsearslts$GSEArslt, qval = 0.01)

In [ ]:
act.me <- TF_Activity(tfs, mDB, neweset, DErslt$Overall)
acts_mat = act.me$all_activities
Activity_heatmap(acts_mat, neweset)

In [ ]:
tf_links = TF_Filter(acts_mat, mDB, miTh = .05, nbins = 8, corMethod = "spearman", DPI = T)

In [ ]:
plot_network(tf_links)


In [ ]:
racipe_results <- sRACIPE::sracipeSimulate(circuit = tf_links, numModels = 200, plots = TRUE)